<a href="https://colab.research.google.com/github/tw107y/unsw_free_rooms/blob/main/unsw_timetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install BeautifulSoup4
from bs4 import BeautifulSoup as bs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install urllib3
import urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://timetable.unsw.edu.au/2023/subjectSearch.html

In [ ]:
# turn a url to raw html with http requests. 
# Pretty slow to run. Try not to run it.
def url_to_raw_html(url: str) -> str:
  http_pool = urllib3.connection_from_url(url)
  response = http_pool.urlopen('GET',url)
  soup = bs(response.data)
  return soup.prettify()

This block stores all room informations

THIS BLOCK IS NOT USED RN BUT COULD BE USEFUL!

Problems: some rooms seem to be missing? E.g when usage is set up turn and learn K-E8-G02 - Science & Engineering G02 shows. But we all know G01 and G03 are identical rooms to G03. Why are those not there? Potential freerooms are missing. 

Improvement: can use the usage tag to filter out lecture rooms rather than using max capacity. 

In [ ]:
main_url = 'https://www.learningenvironments.unsw.edu.au/find-teaching-space?building_name=&room_name=&field_room_capacity_int_value%5Bmin%5D=&field_room_capacity_int_value%5Bmax%5D='
# default is 75 so we get rooms instead of lecture halls. Could go for lower values for smaller rooms.
max_cap = '75'
page_url = '&page='
pg_no = 0
combined_url = main_url + max_cap + page_url + str(pg_no)

# Assume info_scraper is written
while info_scrapper(combined_url):
  pg_no += 1
  combined_url = main_url + max_cap + page_url + str(pg_no)


New plan:

hardcode undergrad t2/postgrad t2

find school on main page: https://timetable.unsw.edu.au/2023/KENSPGRDT2.html

find course on school page

find class time on course page

Problem: some other terms like hexamester is not accounted for

In [ ]:
all_times = []
all_locations = []
ug_schools = []
pg_schools = []
course_codes = set()

In [ ]:
class TimeBlock:
  def __init__(self, day, start, end, location):
    self.day = day
    self.start = start
    self.end = end
    self.location = location

  def __str__(self):
    return f"Day: {self.day}.\nTime: {self.start} - {self.end}.\nLocation: {self.location}."

In [ ]:
def school_scrapper(url, b):
  raw_html = url_to_raw_html(url)
  lines = raw_html.split('\n')
  all_matches = []
  for line in lines:
    nw_line = line.strip()
    if re.search("^[A-Z]{4}$",nw_line):
      all_matches.append(re.findall("^[A-Z]{4}$",nw_line))
  for school in all_matches:
    if b == 1:
      ug_schools.append(school[0])
    else:
      pg_schools.append(school[0])

# school_scrapper('https://timetable.unsw.edu.au/2023/KENSUGRDT2.html')

In [ ]:
def course_scrapper(url):
  raw_html = url_to_raw_html(url)
  lines = raw_html.split('\n')
  all_matches = []
  for line in lines:
    nw_line = line.strip()
    if re.search("^[A-Z]{4}[0-9]{4}$",nw_line):
      all_matches.append(re.findall("^[A-Z]{4}[0-9]{4}$",nw_line))
  for code in all_matches:
    course_codes.add(code[0])

# course_scrapper('https://timetable.unsw.edu.au/2023/KENSUGRDT2ACCT.html')

PROBLEM: FUCKED IF A CLASS HAVE 2 LOCATIONS. NEED TO CHECK FURTHER FOR THAT CASE

Problem: sometimes location might be empty. Need to do a regex check rather than checking whether its online. Currently this is solved when we load the json data but not solved when we run the code for the first time.

In [ ]:
_day = 15
_time = 18
_location = 21

def time_scrapper(url):
  raw_html = url_to_raw_html(url)
  lines = raw_html.split('\n')
  for idx, line in enumerate(lines):
    nw_line = line.strip()
    if re.search("Location",nw_line):
      # FIND A BETTER WAY THAN JUST HARD CODING
      day_line = lines[idx + _day].strip()
      time_line = lines[idx + _time].strip()
      location_line = lines[idx + _location].strip()
      location_info = location_line.split(' ')
      if location_info[-1][1:-1] == 'ONLINE':
        continue
      time = TimeBlock(day_line, time_line[0:5], time_line[8:13], location_info[-1][1:-1])
      all_times.append(time)

# time_scrapper('https://timetable.unsw.edu.au/2023/ACCT5910.html')

In [ ]:
ugt2_url = 'https://timetable.unsw.edu.au/2023/KENSUGRDT2.html'
pgt2_url = 'https://timetable.unsw.edu.au/2023/KENSPGRDT2.html'
school_scrapper(ugt2_url, 1)
school_scrapper(pgt2_url, 0)
ug_base_url = 'https://timetable.unsw.edu.au/2023/KENSUGRDT2' # + SCHL + .html
pg_base_url = 'https://timetable.unsw.edu.au/2023/KENSPGRDT2' # + SCHL + .html

In [ ]:
print(ug_schools)
print(pg_schools)

['ACCT', 'ACTL', 'AERO', 'ANAT', 'ARCH', 'ARTS', 'ATSI', 'AVIA', 'BABS', 'BEES', 'BEIL', 'BENV', 'BINF', 'BIOC', 'BIOM', 'BIOS', 'BLDG', 'CDEV', 'CEIC', 'CHEM', 'CHEN', 'CLIM', 'CODE', 'COMM', 'COMP', 'CRIM', 'CVEN', 'DATA', 'DESN', 'ECON', 'EDST', 'ELEC', 'ENGG', 'FADA', 'FADA', 'FINS', 'FOOD', 'GENC', 'GENE', 'GENL', 'GENS', 'GENY', 'GEOS', 'GMAT', 'HDAT', 'HESC', 'HLTH', 'HUMS', 'IDES', 'INFS', 'INST', 'INTA', 'LAND', 'LAWS', 'MANF', 'MARK', 'MATH', 'MATS', 'MDIA', 'MECH', 'MERE', 'MFAC', 'MGMT', 'MINE', 'MMAN', 'MSCI', 'MTRN', 'MUSC', 'NANO', 'NEUR', 'OPTM', 'PATH', 'PHAR', 'PHCM', 'PHRM', 'PHSL', 'PHTN', 'PHYS', 'PLAN', 'PSCY', 'PSYC', 'PTRL', 'REGZ', 'RISK', 'SCIF', 'SENG', 'SOCW', 'SOLA', 'SOMS', 'SOSS', 'SRAP', 'TABL', 'TELE', 'VISN']
['ACCT', 'ACTL', 'AGSM', 'ARCH', 'AVIA', 'AVIG', 'BABS', 'BEES', 'BENV', 'BINF', 'BIOM', 'BIOS', 'CDEV', 'CEIC', 'CHEM', 'CHEN', 'CLIM', 'COMD', 'COMM', 'COMP', 'CONS', 'CVEN', 'DATA', 'ECON', 'EDST', 'ELEC', 'ENGG', 'FADA', 'FADA', 'FINS', 'FOOD'

In [ ]:
for school in ug_schools:
  complete_url = ug_base_url + school + '.html'
  course_scrapper(complete_url)
for school in pg_schools:
  complete_url = pg_base_url + school + '.html'
  course_scrapper(complete_url)

In [ ]:
print(len(course_codes))

1381


In [ ]:
print(sorted(course_codes))

['ACCT2542', 'ACCT3202', 'ACCT3303', 'ACCT3610', 'ACCT3672', 'ACCT3995', 'ACCT4797', 'ACCT4852', 'ACCT4897', 'ACCT5910', 'ACCT5919', 'ACCT5925', 'ACCT5930', 'ACCT5942', 'ACCT5943', 'ACCT5949', 'ACCT5952', 'ACCT5961', 'ACCT5967', 'ACCT5995', 'ACCT5996', 'ACCT5998', 'ACCT5999', 'ACTL1101', 'ACTL2101', 'ACTL2102', 'ACTL3142', 'ACTL3143', 'ACTL3202', 'ACTL3301', 'ACTL3303', 'ACTL4002', 'ACTL4003', 'ACTL4011', 'ACTL5004', 'ACTL5103', 'ACTL5110', 'ACTL5111', 'ACTL5200', 'ACTL5301', 'ACTL6104', 'ACTL6105', 'AERO3110', 'AGSM6131', 'AGSM6143', 'AGSM6210', 'AGSM6234', 'AGSM6251', 'AGSM6275', 'AGSM6321', 'AGSM9101', 'AGSM9132', 'AGSM9137', 'AGSM9150', 'AGSM9152', 'AGSM9153', 'AGSM9154', 'AGSM9155', 'AGSM9156', 'AGSM9271', 'AGSM9274', 'ANAT1451', 'ANAT1521', 'ANAT2111', 'ANAT2241', 'ANAT2521', 'ANAT3131', 'ARCH1161', 'ARCH1162', 'ARCH1222', 'ARCH1261', 'ARCH1302', 'ARCH1361', 'ARCH7112', 'ARCH7182', 'ARCH7202', 'ARCH7213', 'ARCH7220', 'ARCH7251', 'ARCH7252', 'ARCH7253', 'ARCH7254', 'ARCH7304', 'AR

In [ ]:
print(course_codes)

{'AVIA5006', 'FINS5512', 'MERE5001', 'LAWS3381', 'CHEM6041', 'ARTS2278', 'AVIA2401', 'INFS5777', 'COMP3511', 'EDST2094', 'AVIA4003', 'COMM3030', 'ENGG9744', 'BIOC3671', 'DATA3199', 'MNGT5282', 'ACCT3995', 'CHEM2011', 'ACCT5925', 'CRIM2014', 'JURD7336', 'MSCI1001', 'CHEM4501', 'ARTS2033', 'ARTS4613', 'COMP3151', 'SRAP4052', 'INFS1603', 'PLAN7145', 'SENG3992', 'ARTS3048', 'MATH1251', 'PHCM3001', 'PHYS4145', 'MDIA3093', 'MMAN3200', 'CVEN3502', 'PHYS4143', 'MGMT3101', 'MUSC3801', 'GSOE9340', 'BIOC3111', 'COMP3153', 'SOLA1070', 'MDIA2013', 'ECON6350', 'ARCH7254', 'MATS4503', 'PHRM1021', 'OPTM7302', 'ACCT3672', 'CHEM1821', 'ACTL5301', 'ARTS3821', 'LAWS1021', 'SOLA5057', 'GENC3004', 'ECON1101', 'HESC4561', 'CVEN9521', 'MATH5925', 'ARTS2248', 'MATS4004', 'CVEN4951', 'HDAT9100', 'MUSC4705', 'MDIA5024', 'BIOM9660', 'ARTS1210', 'ARTS2754', 'EDST5888', 'MATH3000', 'COMP4961', 'MERE4951', 'LAWS2383', 'ARTS4840', 'PLAN4004', 'CVEN4963', 'COMP9993', 'MERE9451', 'BIOM9332', 'REST0005', 'BEES2741', 'MA

In [ ]:
course_base_url = 'https://timetable.unsw.edu.au/2023/' # + CODECODE + .html
for course in course_codes:
  complete_url = course_base_url + course + '.html'
  time_scrapper(complete_url)

In [ ]:
print(len(all_times))

5864


In [ ]:
import json

In [ ]:
json_string = json.dumps([obj.__dict__ for obj in all_times])

with open('/content/drive/MyDrive/data/unsw_timetable/time_data', 'w') as file:
    json.dump(json_string, file)

In [ ]:
all_times = []

In [ ]:
raw_json = None
with open('/content/drive/MyDrive/data/unsw_timetable/time_data', 'r') as file:
    raw_json = json.load(file)
all_times_raw = json.loads(raw_json)

for d in all_times_raw:
  if not d['location']:
    continue
  all_times.append(TimeBlock(d['day'], d['start'], d['end'], d['location']))
print(len(all_times))

5653
